In [1]:
import nest_asyncio
nest_asyncio.apply()

from crewai import LLM
from dotenv import load_dotenv
import os

load_dotenv()

llm = LLM(
    model=os.getenv("FIREWORKS_MODEL_NAME"),
    base_url="https://api.fireworks.ai/inference/v1",
    api_key=os.getenv("FIREWORKS_API_KEY")
)

Data directory: /Users/sali/Library/Application Support/crewai_storage


/Users/sali/miniconda3/envs/ML/lib/python3.10/site-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)


Data directory: /Users/sali/Library/Application Support/crewai_storage


In [2]:
from crewai import Agent

agent = Agent(
    role="Sales Call Analysis Specialist",
    goal="Analyze the sales call between buyer and {seller} to identify the key pain points, challenges, objections, insights and areas of improvement.",
    backstory=(
        "You are an expert in analyzing discovery sales calls and identifying key insights."
        "Your goal is to analyze the sales call between buyer and {seller}."
        "Your goal is to identify the pain points and objections, areas of improvement, and potential strategies for future calls."
        ""
    ),
    llm=llm
)

In [3]:
from echo.runner import make_call

test_clients = [
	"Shell",
	"Schneider electric"
]


train_clients = [
	"ICICI bank",
	"Infosys",	
	"University of Illinois",
	"Marks and spencer",
	"Mercedes Benz",
]

clients = test_clients + train_clients


inputs = {
    'call_type': 'discovery', 
    'seller': 'Whatfix',
    'n_competitors': 3
}

In [4]:
import asyncio
import nest_asyncio
nest_asyncio.apply()

In [ ]:
discovery_calls_data = asyncio.run(make_call('discovery', clients, inputs))

In [6]:
import echo.utils as utils
# utils.remove_keys(['demo_transcript', 'demo_analysis', 'demo_features'])

In [ ]:
demo_calls_data = asyncio.run(make_call('demo', clients, inputs))

## Memory Module

In [ ]:
import logging
import os
import shutil
import uuid

from typing import Any, Dict, List, Optional
from chromadb.api import ClientAPI
from abc import ABC, abstractmethod
from chromadb import EmbeddingFunction
from chromadb.api.types import validate_embedding_function
import os
from pathlib import Path

import appdirs
import contextlib
import io

import json
import sqlite3
from typing import Any, Dict, List, Optional
import hashlib

from echo.constants import EMBED_STRING_HASH_KEY

def sha256_hash(text: str) -> str:
    return hashlib.sha256(text.encode()).hexdigest()


class Printer:
    def print(self, content: str, color: Optional[str] = None):
        if color == "purple":
            self._print_purple(content)
        elif color == "red":
            self._print_red(content)
        elif color == "bold_green":
            self._print_bold_green(content)
        elif color == "bold_purple":
            self._print_bold_purple(content)
        elif color == "bold_blue":
            self._print_bold_blue(content)
        elif color == "yellow":
            self._print_yellow(content)
        elif color == "bold_yellow":
            self._print_bold_yellow(content)
        else:
            print(content)

    def _print_bold_purple(self, content):
        print("\033[1m\033[95m {}\033[00m".format(content))

    def _print_bold_green(self, content):
        print("\033[1m\033[92m {}\033[00m".format(content))

    def _print_purple(self, content):
        print("\033[95m {}\033[00m".format(content))

    def _print_red(self, content):
        print("\033[91m {}\033[00m".format(content))

    def _print_bold_blue(self, content):
        print("\033[1m\033[94m {}\033[00m".format(content))

    def _print_yellow(self, content):
        print("\033[93m {}\033[00m".format(content))

    def _print_bold_yellow(self, content):
        print("\033[1m\033[93m {}\033[00m".format(content))



@contextlib.contextmanager
def suppress_logging(
    logger_name="chromadb.segment.impl.vector.local_persistent_hnsw",
    level=logging.ERROR,
):
    logger = logging.getLogger(logger_name)
    original_level = logger.getEffectiveLevel()
    logger.setLevel(level)
    with (
        contextlib.redirect_stdout(io.StringIO()),
        contextlib.redirect_stderr(io.StringIO()),
        contextlib.suppress(UserWarning),
    ):
        yield
    logger.setLevel(original_level)



def db_storage_path():
    app_name = get_project_directory_name()
    app_author = "Echo"

    data_dir = Path(appdirs.user_data_dir(app_name, app_author))
    data_dir.mkdir(parents=True, exist_ok=True)

    print(f"Data directory: {data_dir}")
    return data_dir


def get_project_directory_name():
    project_directory_name = os.environ.get("ECHO_STORAGE_DIR")

    if project_directory_name:
        return project_directory_name
    else:
        cwd = Path.cwd()
        project_directory_name = cwd.name
        return project_directory_name



class BaseRAGStorage(ABC):
    """
    Base class for RAG-based Storage implementations.
    """

    app: Any | None = None

    def __init__(
        self,
        type: str,
        allow_reset: bool = True,
        embedder_config: Optional[Any] = None,
    ):
        self.type = type
        self.allow_reset = allow_reset
        self.embedder_config = embedder_config
        
    @abstractmethod
    def save(self, value: Any, metadata: Dict[str, Any]) -> None:
        """Save a value with metadata to the storage."""
        pass

    @abstractmethod
    def search(
        self,
        query: str,
        limit: int = 3,
        filter: Optional[dict] = None,
        score_threshold: float = 0.35,
    ) -> List[Any]:
        """Search for entries in the storage."""
        pass

    @abstractmethod
    def reset(self) -> None:
        """Reset the storage."""
        pass

    @abstractmethod
    def _generate_embedding(
        self, text: str, metadata: Optional[Dict[str, Any]] = None
    ) -> Any:
        """Generate an embedding for the given text and metadata."""
        pass

    @abstractmethod
    def _initialize_app(self):
        """Initialize the vector db."""
        pass

    def setup_config(self, config: Dict[str, Any]):
        """Setup the config of the storage."""
        pass

    def initialize_client(self):
        """Initialize the client of the storage. This should setup the app and the db collection"""
        pass


class EmbeddingConfigurator:
    def __init__(self):
        self.embedding_functions = {
            "openai": self._configure_openai,
            "azure": self._configure_azure,
            "ollama": self._configure_ollama,
            "vertexai": self._configure_vertexai,
            "google": self._configure_google,
            "cohere": self._configure_cohere,
            "bedrock": self._configure_bedrock,
            "huggingface": self._configure_huggingface
        }

    def configure_embedder(
        self,
        embedder_config: Dict[str, Any] | None = None,
    ) -> EmbeddingFunction:
        """Configures and returns an embedding function based on the provided config."""
        if embedder_config is None:
            return self._create_default_embedding_function()

        provider = embedder_config.get("provider")
        config = embedder_config.get("config", {})
        model_name = config.get("model")

        if isinstance(provider, EmbeddingFunction):
            try:
                validate_embedding_function(provider)
                return provider
            except Exception as e:
                raise ValueError(f"Invalid custom embedding function: {str(e)}")

        if provider not in self.embedding_functions:
            raise Exception(
                f"Unsupported embedding provider: {provider}, supported providers: {list(self.embedding_functions.keys())}"
            )

        return self.embedding_functions[provider](config, model_name)

    @staticmethod
    def _create_default_embedding_function():
        from chromadb.utils.embedding_functions.openai_embedding_function import (
            OpenAIEmbeddingFunction,
        )

        return OpenAIEmbeddingFunction(
            api_key=os.getenv("OPENAI_API_KEY"), 
            model_name="text-embedding-3-small"
        )

    @staticmethod
    def _configure_openai(config, model_name):
        from chromadb.utils.embedding_functions.openai_embedding_function import (
            OpenAIEmbeddingFunction,
        )

        return OpenAIEmbeddingFunction(
            api_key=config.get("api_key") or os.getenv("OPENAI_API_KEY"),
            model_name=config.get("model") or model_name,
        )

    @staticmethod
    def _configure_azure(config, model_name):
        from chromadb.utils.embedding_functions.openai_embedding_function import (
            OpenAIEmbeddingFunction,
        )

        return OpenAIEmbeddingFunction(
            api_key=config.get("api_key"),
            api_base=config.get("api_base"),
            api_type=config.get("api_type", "azure"),
            api_version=config.get("api_version"),
            model_name=config.get("model") or model_name,
        )

    @staticmethod
    def _configure_ollama(config, model_name):
        from chromadb.utils.embedding_functions.ollama_embedding_function import (
            OllamaEmbeddingFunction,
        )

        return OllamaEmbeddingFunction(
            url=config.get("url", "http://localhost:11434/api/embeddings"),
            model_name=config.get("model") or model_name,
        )

    @staticmethod
    def _configure_vertexai(config, model_name):
        from chromadb.utils.embedding_functions.google_embedding_function import (
            GoogleVertexEmbeddingFunction,
        )

        return GoogleVertexEmbeddingFunction(
            model_name=config.get("model") or model_name,
            api_key=config.get("api_key"),
        )

    @staticmethod
    def _configure_google(config, model_name):
        from chromadb.utils.embedding_functions.google_embedding_function import (
            GoogleGenerativeAiEmbeddingFunction,
        )

        return GoogleGenerativeAiEmbeddingFunction(
            model_name=model_name,
            api_key=config.get("api_key"),
        )

    @staticmethod
    def _configure_cohere(config, model_name):
        from chromadb.utils.embedding_functions.cohere_embedding_function import (
            CohereEmbeddingFunction,
        )

        return CohereEmbeddingFunction(
            model_name=config.get("model") or model_name,
            api_key=config.get("api_key"),
        )

    @staticmethod
    def _configure_bedrock(config, model_name):
        from chromadb.utils.embedding_functions.amazon_bedrock_embedding_function import (
            AmazonBedrockEmbeddingFunction,
        )

        return AmazonBedrockEmbeddingFunction(
            session=config.get("session"),
            model_name=config.get("model") or model_name,
        )

    @staticmethod
    def _configure_huggingface(config, model_name):
        from chromadb.utils.embedding_functions.huggingface_embedding_function import (
            HuggingFaceEmbeddingFunction,
        )

        return HuggingFaceEmbeddingFunction(
            api_key=os.getenv("HF_ACCESS_TOKEN"),
            model_name=config.get("model") or model_name,
        )


class RAGStorage(BaseRAGStorage):
    """
    Extends Storage to handle embeddings for memory entries, improving
    search efficiency.
    """

    app: ClientAPI | None = None

    def __init__(self, type, allow_reset=True, embedder_config=None, path=None):
        super().__init__(type, allow_reset, embedder_config)
        self.type = type
        self.allow_reset = allow_reset
        self.path = path
        self._initialize_app()

    def _set_embedder_config(self):
        configurator = EmbeddingConfigurator()
        self.embedder_config = configurator.configure_embedder(self.embedder_config)

    def _initialize_app(self):
        import chromadb
        from chromadb.config import Settings

        self._set_embedder_config()
        chroma_client = chromadb.PersistentClient(
            path=self.path if self.path else f"{db_storage_path()}{os.sep}{self.type}",
            settings=Settings(allow_reset=self.allow_reset),
        )

        self.app = chroma_client

        try:
            self.collection = self.app.get_collection(
                name=self.type, embedding_function=self.embedder_config
            )
        except Exception:
            self.collection = self.app.create_collection(
                name=self.type, embedding_function=self.embedder_config
            )

    def check_text_embedded(self, text: str) -> bool:
        text_hash = sha256_hash(text)
        try:
            filter_criteria = {EMBED_STRING_HASH_KEY: text_hash}
            results = self.collection.get(
                where=filter_criteria, include=["metadatas"]
            )
            return len(results["metadatas"]) > 0
        except Exception as e:
            logging.error(f"Error checking if text is embedded: {str(e)}")
            return False
    
    
    def save(self, value: Any, metadata: Dict[str, Any]) -> None:
        if not hasattr(self, "app") or not hasattr(self, "collection"):
            self._initialize_app()
        
        try:
            self._generate_embedding(value, metadata)
        except Exception as e:
            logging.error(f"Error during {self.type} save: {str(e)}")

    def search(
        self,
        query: str,
        limit: int = 3,
        filter: Optional[dict] = None,
        score_threshold: float = 0.35,
    ) -> List[Any]:
        if not hasattr(self, "app"):
            self._initialize_app()

        try:
            with suppress_logging():
                response = self.collection.query(query_texts=query, n_results=limit)

            results = []
            for i in range(len(response["ids"][0])):
                result = {
                    "id": response["ids"][0][i],
                    "metadata": response["metadatas"][0][i],
                    "context": response["documents"][0][i],
                    "score": response["distances"][0][i],
                }
                if result["score"] >= score_threshold:
                    results.append(result)

            if filter is not None:
                results = [
                    result
                    for result in results
                    if all(
                        [
                            result["metadata"].get(key) == value
                            for key, value in filter.items()
                        ]
                    )
                ]
            
            return results
        except Exception as e:
            logging.error(f"Error during {self.type} search: {str(e)}")
            return []

    def _generate_embedding(self, text: str, metadata: Dict[str, Any]) -> None:  # type: ignore
        if not hasattr(self, "app") or not hasattr(self, "collection"):
            self._initialize_app()

        if self.check_text_embedded(text):
            logging.info(f"Text already embedded in {self.type}")
            return
        metadata = metadata or {}
        metadata[EMBED_STRING_HASH_KEY] = sha256_hash(text)
        self.collection.add(
            documents=[text],
            metadatas=[metadata or {}],
            ids=[str(uuid.uuid4())],
        )

    def reset(self) -> None:
        try:
            shutil.rmtree(f"{db_storage_path()}/{self.type}")
            if self.app:
                self.app.reset()
        except Exception as e:
            if "attempt to write a readonly database" in str(e):
                # Ignore this specific error
                pass
            else:
                raise Exception(
                    f"An error occurred while resetting the {self.type} memory: {e}"
                )

    def _create_default_embedding_function(self):
        from chromadb.utils.embedding_functions.openai_embedding_function import (
            OpenAIEmbeddingFunction,
        )

        return OpenAIEmbeddingFunction(
            api_key=os.getenv("OPENAI_API_KEY"), model_name="text-embedding-3-small"
        )



class LTMSQLiteStorage:
    """
    An updated SQLite storage class for LTM data storage.
    """

    def __init__(
        self, 
        db_type: str = f"long_term_memory_storage.db",
        reset: bool = False,
    ) -> None:
        self.db_path = f"{db_storage_path()}/{db_type}.db"
        self._printer: Printer = Printer()
        self._initialize_db(reset)

    def _initialize_db(self, reset=False):
        """
        Initializes the SQLite database and creates LTM table
        """
        try:
            with sqlite3.connect(self.db_path) as conn:
                if reset:
                    print("Deleting existing table")
                    conn.execute("DROP TABLE IF EXISTS seller_data")
                    conn.commit()
                
                conn.execute('PRAGMA foreign_keys = ON;')
                cursor = conn.cursor()
                cursor.execute(
                    """
                    CREATE TABLE IF NOT EXISTS seller_data (
                        id INTEGER PRIMARY KEY AUTOINCREMENT,
                        seller TEXT,
                        buyer TEXT,
                        call_type TEXT,
                        data JSON,
                        datetime TEXT
                    )
                """
                )

                conn.commit()
        except sqlite3.Error as e:
            self._printer.print(
                content=f"MEMORY ERROR: An error occurred during database initialization: {e}",
                color="red",
            )


    def check_if_exists(self, seller, buyer, call_type, metadata: Dict) -> int:
        """
        Checks if a row exists in the LTM table
        return:
            0: if the row does not exist
            1: if the row exists but metadata does not match
            2: if the row exists and metadata matches
        """
        query = "SELECT * FROM seller_data WHERE seller = ? AND buyer = ? AND call_type = ?"
        with sqlite3.connect(self.db_path) as conn:
            cursor = conn.cursor()
            cursor.execute(query, (seller, buyer, call_type))
            row = cursor.fetchone()
            if row:
                row_metadata: Dict = json.loads(row[3])
                if all(
                    [
                        key in row_metadata
                        for key in metadata.items()
                    ]
                ):
                    return True
        return False
                
        
    def save(
        self,
        seller: str,
        buyer: str,
        call_type: str,
        data: Dict[str, Any]
    ) -> None:
        datetime = utils.get_current_time()
        """Saves data to the LTM table with error handling."""
        record_exists =  self.check_if_exists(seller, buyer, call_type)
        assert record_exists in [0, 1, 2], f"Invalid record_exists value: {record_exists}"
        if record_exists == 2:
            logging.info(f"Record already exists for seller: {seller}, buyer: {buyer}, call_type: {call_type}")
            return 
        else:
            try:
                with sqlite3.connect(self.db_path) as conn:
                    cursor = conn.cursor()
                    if record_exists == 1:
                        query = "UPDATE seller_data SET data = ?, datetime = ? WHERE seller = ? AND buyer = ? AND call_type = ?"
                        cursor.execute(query, (json.dumps(data), datetime, seller, buyer, call_type))
                    
                    else:
                        query = "INSERT INTO seller_data (seller, buyer, call_type, data, datetime) VALUES (?, ?, ?, ?, ?)"
                        cursor.execute(query, (seller, buyer, call_type, json.dumps(data), datetime))
                    conn.commit()
            except sqlite3.Error as e:
                self._printer.print(
                    content=f"MEMORY ERROR: An error occurred while saving to LTM: {e}",
                    color="red",
                )
        return None

    def load(
        self, 
        buyer: str,
        seller: str, 
        call_type: str, 
        data: Dict=None, 
        latest_n: int
    =-1) -> Optional[List[Dict[str, Any]]]:
        """Queries the LTM table by task description with error handling."""
        latest_n = latest_n if latest_n > 0 else 3
        query = 'SELECT seller, buyer, call_type, data FROM seller_data WHERE seller = ? AND buyer = ? AND call_type = ? '
        params = [seller, buyer, call_type]
        if data:
            for key, value in data.items():
                query += f' AND json_extract(data, \'$.{key}\') = ?'
                params.append(value)
        
        query += f' ORDER BY datetime DESC LIMIT {latest_n}'
        
        try:
            rows = self.run_query(query, params)
            if rows:
                return [
                    {
                        "seller": row[0],
                        "buyer": row[1],
                        "call_type": row[2],
                        "data": json.loads(row[3]),
                    }
                    for row in rows
                ]
        except Exception as e:
            self._printer.print(
                content=f"MEMORY ERROR: An error occurred while querying LTM: {e}",
                color="red",
            )
        return None
    
    
    def run_query(
        self, 
        query: str, 
        params: List[str] = None
    ) -> Optional[List[Dict[str, Any]]]:
        """Queries the LTM table by task description with error handling."""
        if not params:
            params = []
            
        with sqlite3.connect(self.db_path) as conn:
            cursor = conn.cursor()
            cursor.execute(query, params)
            rows = cursor.fetchall()
            if rows:
                return [r for r in rows]
        
        return []
        
    def reset(
        self,
    ) -> None:
        """Resets the LTM table with error handling."""
        try:
            with sqlite3.connect(self.db_path) as conn:
                cursor = conn.cursor()
                cursor.execute("DELETE FROM long_term_memories")
                conn.commit()

        except sqlite3.Error as e:
            self._printer.print(
                content=f"MEMORY ERROR: An error occurred while deleting all rows in LTM: {e}",
                color="red",
            )
        return None

## Using Memory

#### Memory Types

- Deal Summary

In [9]:
from echo.step_templates.discovery import get_client_data_to_embed as get_discovery_data_to_embed
from echo.step_templates.demo import get_client_data_to_embed as get_demo_data_to_embed

from echo.step_templates.discovery import get_client_data_to_save as get_discovery_data_to_save
from echo.step_templates.demo import get_client_data_to_save as get_demo_data_to_save


buyer_data_to_embed = dict(discovery={}, demo={})
buyer_data_to_save = dict(discovery={}, demo={})
buyer_data_to_embed['discovery'] = {c: get_discovery_data_to_embed(c, 'buyer') for c in train_clients}
buyer_data_to_embed['demo'] = {c: get_demo_data_to_embed(c, 'buyer') for c in train_clients}

buyer_data_to_save['discovery'] = {c: get_discovery_data_to_save(c, 'buyer') for c in train_clients}
buyer_data_to_save['demo'] = {c: get_demo_data_to_save(c, 'buyer') for c in train_clients}

seller_data_to_save = dict(discovery={}, demo={})
seller_data_to_save['discovery'] = {c: get_discovery_data_to_save(c, 'seller') for c in train_clients}
seller_data_to_save['demo'] = {c: get_demo_data_to_save(c, 'seller') for c in train_clients}

In [10]:
buyer_rag_storage = RAGStorage(type="buyer_data")
seller_sqlite_storage = LTMSQLiteStorage(db_type='seller_data', reset=True)
buyer_sqlite_storage = LTMSQLiteStorage(db_type='buyer_data', reset=True)

Data directory: /Users/sali/Library/Application Support/echo_storage
Data directory: /Users/sali/Library/Application Support/echo_storage
Deleting existing table
Data directory: /Users/sali/Library/Application Support/echo_storage
Deleting existing table


In [11]:
print(json.dumps(seller_data_to_save, indent=2))

{
  "discovery": {
    "ICICI bank": {
      "Seller Research": {
        "name": "Whatfix",
        "website": "https://whatfix.com/",
        "description": "Whatfix is a digital adoption platform that provides organizations with a no-code editor to create in-app guidance on any application that looks 100% native.",
        "industry": "Software as a Service (SaaS)",
        "products": [
          "Digital Adoption Platform",
          "Analytics",
          "In-app Guidance",
          "Simulation-based Training",
          "Application Analytics"
        ],
        "solutions": [
          "Digital Transformation",
          "Change Management",
          "Performance Support",
          "User Adoption",
          "Remote Training"
        ],
        "use_cases": [
          "Supporting change and digital transformation efforts",
          "On-demand employee training, onboarding, and performance support",
          "Customer Onboarding",
          "Employee Onboarding",
         

In [12]:
import time

seller = 'Whatfix'
for call_type in seller_data_to_save:
    for buyer in seller_data_to_save[call_type]:
        data = seller_data_to_save[call_type][buyer]
        timestamp = time.strftime('%Y-%m-%d %H:%M:%S')
        seller_sqlite_storage.save(seller, buyer, call_type, data, timestamp)

In [13]:
rows = seller_sqlite_storage.run_query('SELECT * FROM seller_data')
print(rows[0][4])

query = '''SELECT * FROM seller_data WHERE data = ?'''
rows = seller_sqlite_storage.run_query(query=query, params=[rows[0][4]])
print(len(rows))

{"Seller Research": {"name": "Whatfix", "website": "https://whatfix.com/", "description": "Whatfix is a digital adoption platform that provides organizations with a no-code editor to create in-app guidance on any application that looks 100% native.", "industry": "Software as a Service (SaaS)", "products": ["Digital Adoption Platform", "Analytics", "In-app Guidance", "Simulation-based Training", "Application Analytics"], "solutions": ["Digital Transformation", "Change Management", "Performance Support", "User Adoption", "Remote Training"], "use_cases": ["Supporting change and digital transformation efforts", "On-demand employee training, onboarding, and performance support", "Customer Onboarding", "Employee Onboarding", "Software Adoption"]}, "Seller Pricing": {"pricing_models": [{"type": "Free", "description": "Free Plan with 30 credits", "price": "Free", "duration": "N/A"}, {"type": "Basic", "description": "Basic Plan with limited features", "price": "$39 per month", "duration": "Mont

In [14]:
for r in rows:
    print(r[4])

{"Seller Research": {"name": "Whatfix", "website": "https://whatfix.com/", "description": "Whatfix is a digital adoption platform that provides organizations with a no-code editor to create in-app guidance on any application that looks 100% native.", "industry": "Software as a Service (SaaS)", "products": ["Digital Adoption Platform", "Analytics", "In-app Guidance", "Simulation-based Training", "Application Analytics"], "solutions": ["Digital Transformation", "Change Management", "Performance Support", "User Adoption", "Remote Training"], "use_cases": ["Supporting change and digital transformation efforts", "On-demand employee training, onboarding, and performance support", "Customer Onboarding", "Employee Onboarding", "Software Adoption"]}, "Seller Pricing": {"pricing_models": [{"type": "Free", "description": "Free Plan with 30 credits", "price": "Free", "duration": "N/A"}, {"type": "Basic", "description": "Basic Plan with limited features", "price": "$39 per month", "duration": "Mont

In [15]:
seller_sqlite_storage.load(buyer=buyer, seller=seller, call_type='demo')

[{'seller': 'Whatfix',
  'buyer': 'Mercedes Benz',
  'call_type': 'demo',
  'data': {'Demo Features': [{'name': 'In-app Guidance',
     'description': 'Provides interactive guides or flows that fit the specific needs of software applications, guiding users through with personalized support.'},
    {'name': 'Real-time Feedback and Analytics',
     'description': 'Provides comprehensive metrics and analytics to help make data-driven decisions, tracking user adoption, time-to-competency, and other key metrics.'},
    {'name': 'Mirror Feature',
     'description': 'Provides real-time feedback and analytics to measure the success of any digital adoption initiative.'},
    {'name': 'Customizable Platform',
     'description': 'Designed to be highly customizable and can integrate with a wide range of software applications, including custom-built ones.'},
    {'name': 'Personalized Guidance and Support',
     'description': 'Provides personalized guidance and support to users, helping them to 

In [17]:
from tqdm.asyncio import tqdm
import time


for call_type in buyer_data_to_embed:
    for buyer in tqdm(buyer_data_to_embed[call_type], desc=f"Saving {call_type} data"):
        data_txt = buyer_data_to_embed[call_type][buyer]
        data_json: Dict = buyer_data_to_save[call_type][buyer]
        timestamp = time.strftime('%Y-%m-%d %H:%M:%S')
        buyer_sqlite_storage.save(seller, buyer, call_type, data_json, timestamp)
        
        print("Call Type: ", call_type, " Buyer: ", buyer, "Seller: ", seller)
        buyer_rag_storage.save(data_txt, metadata={'seller': seller, 'buyer': buyer, 'call_type': call_type})
        print("--------------------")

Saving discovery data:   0%|          | 0/5 [00:00<?, ?it/s]

Call Type:  discovery  Buyer:  ICICI bank Seller:  Whatfix
--------------------


Saving discovery data:  20%|██        | 1/5 [00:38<02:35, 38.81s/it]

Call Type:  discovery  Buyer:  Infosys Seller:  Whatfix


Saving discovery data:  40%|████      | 2/5 [00:39<00:49, 16.54s/it]

...

--------------------


Saving discovery data: 100%|██████████| 5/5 [00:44<00:00,  8.82s/it]


Call Type:  discovery  Buyer:  Marks and spencer Seller:  Whatfix
--------------------
Call Type:  discovery  Buyer:  Mercedes Benz Seller:  Whatfix
--------------------


Saving demo data: 100%|██████████| 5/5 [00:00<00:00, 142.10it/s]

Call Type:  demo  Buyer:  ICICI bank Seller:  Whatfix
--------------------
Call Type:  demo  Buyer:  Infosys Seller:  Whatfix
--------------------
Call Type:  demo  Buyer:  University of Illinois Seller:  Whatfix
--------------------
Call Type:  demo  Buyer:  Marks and spencer Seller:  Whatfix
--------------------
Call Type:  demo  Buyer:  Mercedes Benz Seller:  Whatfix
--------------------
